In [8]:
import os
import json
import pandas as pd

In [9]:
results_path = os.path.join(os.getcwd(), "regioni_sidarthe") #, "sidarthe", "Italy")
experiments = [dir for dir in os.listdir(results_path) if os.path.isdir(os.path.join(results_path, dir))]
print(results_path)

exps = {}
for exp_dir in experiments:
    json_path = os.path.join(results_path, exp_dir, "final.json")
    settings_json_path = os.path.join(results_path, exp_dir, "settings.json")
    if os.path.exists(json_path):
        with open(json_path) as json_file:
            exps[exp_dir] = json.load(json_file)
        with open(settings_json_path) as json_file:
            exps[exp_dir]['settings'] = json.load(json_file)

d:\GitHub\covid-tools\regioni_sidarthe


In [10]:
momentum_counts = {
    False: 0,
    True: 0
}

for exp_id, exp in exps.items():
    momentum = exp['settings']['momentum']
    momentum_counts[momentum] += 1

print(momentum_counts)


{False: 441, True: 6}


In [11]:
validation_key = "rmse"

df = pd.DataFrame()
 
#for exp_name, exp in exps.items():
def selector(k):
    tot_loss = 0.
    for loss in "dehrt":   
        item = exps[k]["val_risks"][f"{loss}_rmse"]
        tot_loss += item[0] if isinstance(item, list) else item
    return tot_loss

best_exp = min(exps, key=selector)

print(f"Best exp name: {best_exp}")
print("Settings used: ")
with open(os.path.join(results_path, best_exp, "settings.json")) as settings_file:
    for line in settings_file.readlines():
        print(line, end='')

Best exp name: ccf74add-e505-4982-8ace-e5930fd66bab
Settings used: 
{
    "started": "19/June/2020 10:54:49",
    "region": "Italy",
    "learning_rates": {
        "alpha": 1e-05,
        "beta": 1e-05,
        "gamma": 1e-05,
        "delta": 1e-05,
        "epsilon": 1e-05,
        "theta": 1e-07,
        "xi": 1e-05,
        "eta": 1e-05,
        "mu": 1e-05,
        "nu": 1e-05,
        "tau": 1e-07,
        "lambda": 1e-05,
        "kappa": 1e-05,
        "zeta": 1e-05,
        "rho": 1e-05,
        "sigma": 1e-05
    },
    "target_weights": {
        "d_weight": 3.7600219703046913,
        "r_weight": 14.668337795721268,
        "t_weight": 5.4858988075610435,
        "h_weight": 87.94665542551087
    },
    "train_size": 46,
    "val_len": 20,
    "der_1st_reg": 4775.74012784665,
    "t_inc": 1.0,
    "momentum": false,
    "m": null,
    "a": null,
    "integrator": "Heun",
    "loss_type": "rmse",
    "initial_values": {
        "alpha": [
            0.57,
            0.57,

In [12]:

import torch
# get references
references = {}
param_keys = ['alpha', 'beta', 'gamma', 'delta', 'epsilon', 'theta', 'xi', 'eta', 'mu', 'nu', 'tau', 'lambda', 'kappa', 'zeta', 'rho', 'sigma']
ref_df = pd.read_csv(os.path.join(os.getcwd(), "regioni", "sidarthe_results.csv"))
for key in 'sidarthe':
    references[key] = ref_df[key].tolist()

for key in ["r0", "h_detected"]:
    references[key] = ref_df[key].tolist()

for key in param_keys:
    references[key] = ref_df[key].tolist()

for k,v in references.items():
    references[k] = torch.tensor(v)

In [13]:
#extract targets

from utils.data_utils import select_data

df_file = os.path.join(os.getcwd(), "COVID-19", "dati-andamento-nazionale", "dpc-covid19-ita-andamento-nazionale.csv")
area = ["ITA"]
area_col_name = "stato"  # "Country/Region"

groupby_cols = ["data"]  # ["Date"]

d_col_name = "isolamento_domiciliare"
r_col_name = "ricoverati_con_sintomi"
t_col_name = "terapia_intensiva"
h_detected_col_name = "dimessi_guariti"
e_col_name = "deceduti"  # "Fatalities"

x_target, d_target = select_data(df_file, area, area_col_name, d_col_name, groupby_cols, file_sep=",")
_, y_target = select_data(df_file, area, area_col_name, "totale_positivi", groupby_cols, file_sep=",")
_, r_target = select_data(df_file, area, area_col_name, r_col_name, groupby_cols, file_sep=",")
_, t_target = select_data(df_file, area, area_col_name, t_col_name, groupby_cols, file_sep=",")
_, h_detected_target = select_data(df_file, area, area_col_name, h_detected_col_name, groupby_cols, file_sep=",")
_, e_target = select_data(df_file, area, area_col_name, e_col_name, groupby_cols, file_sep=",")

initial_len = len(y_target)
tmp_d, tmp_r, tmp_t, tmp_h, tmp_e = [], [], [], [], []
for i in range(initial_len):
    if y_target[i] > 0:
        tmp_d.append(d_target[i])
        tmp_r.append(r_target[i])
        tmp_t.append(t_target[i])
        tmp_h.append(h_detected_target[i])
        tmp_e.append(e_target[i])
d_target = tmp_d
r_target = tmp_r
t_target = tmp_t
h_detected_target = tmp_h
e_target = tmp_e

targets = {
    "d": d_target,
    "r": r_target,
    "t": t_target,
    "h_detected": h_detected_target,
    "e": e_target
}

for k,v in targets.items():
    targets[k] = torch.tensor(v, dtype=torch.float32)

In [14]:
def rmse_loss(target, hat):
    return torch.sqrt(
        0.5 * torch.mean(
            torch.pow(target - hat, 2)
        )
    )

losses = {}
tot_loss = 0.
for k,v in targets.items():
    loss = rmse_loss(targets[k], references[k][0:targets[k].size(0)])
    losses[k] = loss
    tot_loss += loss

our_losses = {}
for loss in "dehrt":   
        item = exps[best_exp]["test_risks"][f"{loss}_rmse"]
        our_losses[loss] = item[0] if isinstance(item, list) else item

print(losses)

print(f"Their total loss: {tot_loss}")
print(f"Our total loss: {selector(best_exp)}")

{'d': tensor(39365.4023), 'r': tensor(13762.4814), 't': tensor(1717.9193), 'h_detected': tensor(35828.5469), 'e': tensor(13273.6357)}
Their total loss: 103947.9765625
Our total loss: 24935.70175787406
